# Analyze Vector Tile data

Inputs from
- [geojson spec](https://tools.ietf.org/pdf/rfc7946.pdf)
- [mapbox spec](https://docs.mapbox.com/vector-tiles/specification/)
- https://github.com/tilezen/mapbox-vector-tile

In [3]:
import mapbox_vector_tile
import math
import requests
import io
import pprint
from shapely.geometry import Point
from IPython.display import GeoJSON, display

class VectorTile(object):
    BASE_URL = "https://vectortiles10.geo.admin.ch/mbtiles/ch.swisstopo.leichte-basiskarte.vt/v006/{z}/{x}/{y}.pbf"
    EXTENT = 4096
    SRID_LNGLAT = 4326
    SRID_SPHERICAL_MERCATOR = 3857

    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
        self.x0, self.y0 = self.tile_ul(self.x, self.y+1, self.z)
        self.x_max, self.y_max = self.tile_ul(self.x+1, self.y, self.z)
        
        pbf = self.get(self.x, self.y, self.z)
        decoded = self.decode(pbf)
        self.rawdata = self.sanitize(decoded)
        self.data = self.reproject(self.rawdata)
        
    
    def get(self, x, y, z):
        req_pbf = requests.get(VectorTile.BASE_URL.format(x=x, y=y, z=z))
        pbf = io.BytesIO(req_pbf.content)
        return pbf
        
    def decode(self, pbf):
        data = mapbox_vector_tile.decode(pbf.getbuffer())
        return data

    def sanitize(self, data):
        for topic in data.keys():
            data[topic]["type"] = "FeatureCollection"
            for feature in data[topic]["features"]:
                feature["type"] = "Feature"
        return data

    def reproject(self, data):
        for topic in data.keys():
            for feature in data[topic]["features"]:
                feature_type = feature["geometry"]["type"]
#                  {'coordinates': [[-80, 3270], [-49, 3275]],
#                             'type': 'LineString'},
                if feature_type == "LineString":
                    coords_merc = []
                    for coord in feature["geometry"]["coordinates"]:
                        tmp = self.tile_coords_to_spherical_mercator(*coord)
                        coords_merc.append(tmp)
                    feature["geometry"]["coordinates"] = coords_merc
                elif feature_type == "Polygon":
                    coords_merc = []
                    for coord in feature["geometry"]["coordinates"][0]:
#                         print(coord)
                        tmp = self.tile_coords_to_spherical_mercator(*coord)
#                         print(tmp)
                        coords_merc.append(tmp)
                    feature["geometry"]["coordinates"] = [coords_merc]
                elif feature_type == "MultiPolygon":
                    coords_merc = []
                    for poly in feature["geometry"]["coordinates"]:
                        coord_poly_merc = []
                        for coord in poly[0]:
#                             print(coord)
                            tmp = self.tile_coords_to_spherical_mercator(*coord)
                            coord_poly_merc.append(tmp)
                        coords_merc.append([coord_poly_merc])
                    feature["geometry"]["coordinates"] = coords_merc
        return data
    
    def tile_coords_to_spherical_mercator(self, tx, ty, MVT_EXTENT=VectorTile.EXTENT):
        
        x_span = self.x_max - self.x0
        y_span = self.y_max - self.y0

        # tx = int( (x_merc - x0) * MVT_EXTENT / x_span )
        x_merc = tx * x_span / MVT_EXTENT + self.x0
        y_merc = ty * y_span / MVT_EXTENT + self.y0
        
        return [x_merc, y_merc]

    def tile_ul(self, x, y, z):
        """get the upper left lon/lat coordinates of the tile with coordinates x/y/z"""
        n = 2.0 ** z
        lon_deg = x / n * 360.0 - 180.0
        lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
        lat_deg = math.degrees(lat_rad)
        return lon_deg,lat_deg
    

vt = VectorTile(2130,1442,12)
print(vt.data.keys())

dict_keys(['hydrology-ln', 'hydrology-pn', 'landcover', 'landuse', 'railtraffic', 'roadtraffic', 'settlement', 'settlement-name', 'terrain-pt-name', 'territory'])


In [4]:
GeoJSON(vt.data['settlement'])

<IPython.display.GeoJSON object>

In [5]:
GeoJSON(vt.data['territory'])

<IPython.display.GeoJSON object>

In [6]:
GeoJSON(vt.data['landcover'])

<IPython.display.GeoJSON object>

In [7]:
GeoJSON(vt.data['roadtraffic'])

<IPython.display.GeoJSON object>

In [8]:
GeoJSON(vt.data['railtraffic'])

<IPython.display.GeoJSON object>

In [9]:
GeoJSON(vt.data['hydrology-ln'])

<IPython.display.GeoJSON object>

In [10]:
from IPython.display import GeoJSON, display
GeoJSON({
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [-118.4563712, 34.0163116]
    }
})

<IPython.display.GeoJSON object>